In [39]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
TICKER = 'SPY'
INTERVAL='1d'

# set period based on interval
if INTERVAL == '1h':
    PERIOD = '730d'
else:
    PERIOD = 'max'
LOOKBACK = 10000

FAST = list(range(3, 200))
SLOW = list(range(4, 200))

def get_data(ticker=TICKER, lookback=LOOKBACK, interval=INTERVAL):

    # get data at interval you want
    df = yf.download(ticker, interval=interval, auto_adjust=False)
    df.columns = df.columns.get_level_values(0)

    # reset the index to make plots prettier
    df = df.reset_index(drop=True)

    # only return the subset of data you are interested in
    return df.iloc[-lookback:, :]

def add_moving_averages(df, fast, slow):
    df[f'{fast}_ma'] = df['Close'].ewm(span=fast).mean()
    df[f'{slow}_ma'] = df['Close'].ewm(span=slow).mean()

    return df.dropna()

def add_strategy(df, fast, slow):
    
    # long when fast > slow, short otherwise
    df['Strategy'] = np.where(df[f'{fast}_ma'] > df[f'{slow}_ma'], 1, -1)
    df['Strategy'] = df['Strategy'].shift(1)
    return df

def test_strategy(df):
    df['Asset_Returns'] = (1 + df['Close'].pct_change()).cumprod() - 1
    df['Strategy_Returns'] = (1 + df['Close'].pct_change() * df['Strategy']).cumprod() -1

    plt.plot(df['Strategy_Returns'])    
    return df.dropna()

def main():
    base = get_data()
    plt.figure(figsize=(12, 8))
    legend_list = []

    best_returns = -np.inf
    best_params = None

    for s in SLOW:
        for f in FAST:
            if f < s:
                df = base.copy()
                df = add_moving_averages(df, f, s)
                df = add_strategy(df, f, s)
                df = test_strategy(df)
                legend_list.append(f'{f}, {s} cross')

                returns = float(df['Strategy_Returns'].iloc[-1])
                if returns > best_returns:
                    best_returns = returns
                    best_params = [f, s]

    plt.title(f'Optimal Combination: {best_params}')
    return df

main()

[*********************100%***********************]  1 of 1 completed
